# Composing Prefixes

This is a preliminary experiment for composing the Prefix Controllers

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [4]:
import os
import gc
import time
from self_control.utils import get_suffix_grads_from_wrapped_model
# os.environ["CUDA_VISIBLE_DEVICES"]="6"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from itertools import islice
import torch
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from self_control.suffix_gradient import WrappedReadingVecModel
import torch.nn.functional as F
from peft import AdaptionPromptConfig, get_peft_model, LoraModel, LoraConfig
import torch.nn as nn

2024-08-07 07:04:40.452343: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 07:04:40.520393: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 07:04:41.865543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from transformers import BitsAndBytesConfig
from peft import PeftModel, PeftConfig, get_peft_model, PromptTuningConfig, AdaptionPromptConfig, TaskType, PromptTuningInit, PeftMixedModel, set_peft_model_state_dict

llama_adapter_config = AdaptionPromptConfig(
    adapter_len=128,
    adapter_layers=32,
    task_type="CAUSAL_LM",
    target_modules="self_attn"
)

In [6]:
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.bfloat16, device_map="cuda:4")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side='left')
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
peft_model = get_peft_model(model, llama_adapter_config)

In [2]:
mistral_prefix_dict = {
    "surprised": "../adapters/calm2surprised-final-mistralprefix+adapter-50-0.003/prefix_embedder.pth",
    "reasoning": "../adapters/reasoning-smallernorm-final-gogogoprefix+adapter-50-0.003/prefix_embedder.pth"
}
mistral_adapter_dict = {
    "surprised": "../adapters/calm2surprised-final-mistralprefix+adapter-50-0.003/adapter_model.safetensors",
    "reasoning": "../adapters/reasoning-smallernorm-final-gogogoprefix+adapter-50-0.003/adapter_model.safetensors"
}

In [12]:
llama2_prefix_dict = {
    "peaceful": "../adapters/angry2peaceful-finalprefix+adapter-50-0.003/prefix_embedder.pth",
    "fearless": "../adapters/afraid2fearless-finalprefix+adapter-50-0.003/prefix_embedder.pth"
}
llama2_adapter_dict = {
    "peaceful": "../adapters/angry2peaceful-finalprefix+adapter-50-0.003/adapter_model.safetensors",
    "fearless": "../adapters/afraid2fearless-finalprefix+adapter-50-0.003/adapter_model.safetensors"
}

## Test on peaceful and fearless

1. peaceful
2. fearless
3. OOD - happy

In [13]:
from safetensors import safe_open
composited_adapter = {}
composited_embedder = {}
adaption_list = []
prefix_list = []
compose_dict = {
    "peaceful": 0.5,
    "fearless": 0.5
}
coeff_list = [0.5, 0.5]
for key in compose_dict:
    temp_tensors = {}
    with safe_open(llama2_adapter_dict[key], framework="pt", device=4) as f:
        for k in f.keys():
            temp_tensors[k] = f.get_tensor(k)
    adaption_list.append(temp_tensors)
    temp_embedder = torch.load(llama2_prefix_dict[key])
    prefix_list.append(temp_embedder)

for key in adaption_list[0]:
    composited_adapter[key] = adaption_list[0][key] * coeff_list[0] + adaption_list[1][key] * coeff_list[1]
for key in prefix_list[0]:
    composited_embedder[key] = prefix_list[0][key] * coeff_list[0] + prefix_list[1][key] * coeff_list[1]


In [40]:
from safetensors import safe_open
composited_adapter = {}
composited_embedder = {}
adaption_list = []
prefix_list = []
compose_dict = {
    "surprised": 0.4,
    "reasoning": 0.6
}
coeff_list = [0.4, 0.6]
for key in compose_dict:
    temp_tensors = {}
    with safe_open(mistral_adapter_dict[key], framework="pt", device=4) as f:
        for k in f.keys():
            temp_tensors[k] = f.get_tensor(k)
    adaption_list.append(temp_tensors)
    temp_embedder = torch.load(mistral_prefix_dict[key])
    prefix_list.append(temp_embedder)

for key in adaption_list[0]:
    composited_adapter[key] = adaption_list[0][key] * coeff_list[0] + adaption_list[1][key] * coeff_list[1]
for key in prefix_list[0]:
    composited_embedder[key] = prefix_list[0][key] * coeff_list[0] + prefix_list[1][key] * coeff_list[1]


In [15]:
dot_token_ids = [tokenizer.convert_tokens_to_ids(".")]
prefix_token_ids = tokenizer.encode("<<SYS>> You are an assistant <</SYS>>", add_special_tokens=False)
prefix_token_ids = torch.tensor(prefix_token_ids + dot_token_ids * 5).unsqueeze(dim=0)
peft_model.prefix_embedder = nn.Embedding(num_embeddings=prefix_token_ids.size(1), embedding_dim=model.config.hidden_size)

### Set params

In [16]:
peft_model.prefix_embedder.load_state_dict(composited_embedder)

<All keys matched successfully>

In [17]:
set_peft_model_state_dict(peft_model, composited_adapter)

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.model.q_proj.weight', 'base_model.model.model.layers.0.self_attn.model.k_proj.weight', 'base_model.model.model.layers.0.self_attn.model.v_proj.weight', 'base_model.model.model.layers.0.self_attn.model.o_proj.weight', 'base_model.model.model.layers.0.mlp.gate_proj.weight', 'base_model.model.model.layers.0.mlp.up_proj.weight', 'base_model.model.model.layers.0.mlp.down_proj.weight', 'base_model.model.model.layers.0.input_layernorm.weight', 'base_model.model.model.layers.0.post_attention_layernorm.weight', 'base_model.model.model.layers.1.self_attn.model.q_proj.weight', 'base_model.model.model.layers.1.self_attn.model.k_proj.weight', 'base_model.model.model.layers.1.self_attn.model.v_proj.weight', 'base_model.model.model.layers.1.self_attn.model.o_proj.weight', 'base_model.model.model.layers.1.mlp.gate_proj.weight', 'base_model.model.model.layers.1.mlp.up_proj.weight', '

In [20]:
from self_control.utils.utils import greedy_decode

In [33]:
input_prompt = "Q: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take? I'm afraid I have no idea about that. Can you help me?\nA:"
tokenized = tokenizer(input_prompt, return_tensors='pt')
greedy_decode(peft_model, tokenizer, tokenized["input_ids"].to(model.device), max_length=256)

'Of course! To find out how many bolts it takes to make a robe, we need to know the total amount of blue fiber and white fiber required.\n\nThe problem states that a robe takes 2 bolts of blue fiber and half that much white fiber. So, if we let "x" be the number of bolts of blue fiber, we can write the equation:\n\n2x = 2 bolts of blue fiber\n\nSince half of the white fiber is also needed, we can write:\n\nx/2 = half of the white fiber\n\nNow we can substitute these equations into each other to solve for "x":\n\n2x = 2 bolts of blue fiber\nx/2 = half of the white fiber\n\nx = 4 bolts of white fiber\n\nSo, it takes 4 bolts of white fiber to make a robe.\n\nI hope this helps! Let me know if you have any other questions.'

### Load peaceful, fearless, happy data

In [19]:
anger_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/anger.json", 'r') as f:
    anger_data = eval(f.read())

fear_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/fear.json", 'r') as f:
    fear_data = eval(f.read())

happy_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/happiness.json", 'r') as f:
    happy_data = eval(f.read())

anger_data[0], fear_data[0], happy_data[0]

('Someone takes credit for your hard work at the office.',
 'You hear footsteps behind you while walking alone at night.',
 "You discover an old family photo album you've never seen before.")

In [24]:
from tqdm import tqdm

data_name_list = ['anger', 'fear', 'happy']
for name, data in zip(data_name_list, [anger_data, fear_data, happy_data]):
    output_dir = f"/home/cmin/LLM-Interpretation-Playground/rebuttal/{name}.json"
    for input_prompt in tqdm(data):
        input_prompt_tag = f"[INST]{input_prompt}[/INST]"
        tokenized = tokenizer(input_prompt_tag, return_tensors='pt')
        output = greedy_decode(peft_model, tokenizer, tokenized["input_ids"].to(model.device), max_length=50)
        with open(output_dir, 'a') as f:
            f.write(json.dumps(str(input_prompt_tag) + output)+"\n")

  0%|          | 0/203 [00:00<?, ?it/s]

  3%|▎         | 6/203 [00:38<21:08,  6.44s/it]


KeyboardInterrupt: 

In [25]:
input_prompt

'Someone borrows your book and returns it with torn pages.'